In [97]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import csv
import statsmodels.api as sm
from sklearn.feature_selection import SelectKBest,f_regression
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.feature_selection import VarianceThreshold

In [98]:
import os
print(os.getcwd())

d:\IML ASSIGNMENT 2\CODE


In [99]:
df = pd.read_csv('train.csv')

In [100]:
df.drop_duplicates(inplace=True)

In [101]:
X = df.loc[:, df.columns != 'price_doc']
y = df[['price_doc']]

In [102]:
object_cols = X.select_dtypes(include=['object']).columns

In [103]:
label = LabelEncoder()
for col in object_cols:
    X[col] = label.fit_transform(X[col].astype(str))

C:\Users\Irtiza\AppData\Local\Temp\ipykernel_1032\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_1032\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = label.fit_transform(X[col].astype(str))
C:\Users\Irtiza\AppData\Local\Temp\ipykernel_1032\2966562952.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

In [104]:
import numpy as np

X = np.log1p(X)

In [105]:
X.shape

(178661, 271)

In [1]:
num_cols = X.select_dtypes(include=['int64','float64']).columns

NameError: name 'X' is not defined

In [107]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler,RobustScaler

scaler = RobustScaler()
X[num_cols] = scaler.fit_transform(X[num_cols])

In [108]:
X.shape

(178661, 271)

In [109]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=70)
# X = pca.fit_transform(X)

In [110]:
# from sklearn.preprocessing import PolynomialFeatures

# poly = PolynomialFeatures(degree=3)
# X = poly.fit_transform(X)

In [111]:
X.shape

(178661, 271)

In [112]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

In [113]:
df_test = pd.read_csv('test.csv')

In [114]:
df_test = df_test.drop(['row ID'], axis=1)

In [115]:
label_test = LabelEncoder()
for col in object_cols:
    df_test[col] = label_test.fit_transform(df_test[col].astype(str))


In [116]:
df_test.shape

(77789, 271)

In [117]:
df_test = np.log1p(df_test)

In [118]:
df_test[num_cols] = scaler.transform(df_test[num_cols])

In [119]:
# df_test = pca.transform(df_test)

In [120]:
# df_test = poly.transform(df_test)

In [121]:
df_test.shape

(77789, 271)

In [122]:
#convert all int64 to float32
X = X.astype('float32')
df_test = df_test.astype('float32')

In [123]:
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l1, l2
from keras.optimizers import RMSprop,Adam,Adamax
from keras.callbacks import EarlyStopping

In [129]:
# get the number of input features in X and assign to n_features
n_features = X.shape[1]
# Create a Sequential model
model = Sequential()
#kernel_regularizer=l2(0.01))
# Add the first hidden layer with 10 neurons and specify the input shape
model.add(Dense(150, input_dim=n_features, activation='sigmoid'))
# Add the second hidden layer with 5 neurons
model.add(Dense(100, activation='relu'))
# Add the third hidden layer with 3 neurons
model.add(Dense(50, activation='swish'))

model.add(Dense(1, activation='linear'))

# model.add(Dense(256, input_dim=n_features,activation='relu'))
# model.add(Dense(128, activation='sigmoid'))
# model.add(Dense(64, activation='swish'))
# model.add(Dense(32, activation='sigmoid'))
# model.add(Dense(16, activation='relu'))
# model.add(Dense(8, activation='relu'))
# model.add(Dense(4, activation='relu'))
# model.add(Dense(1, activation='linear'))






# Compile the model
model.compile(optimizer=Adamax(), loss='mean_squared_error') #change optimizer

# Print the model summary
print(model.summary())


Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 150)               40800     
                                                                 
 dense_23 (Dense)            (None, 100)               15100     
                                                                 
 dense_24 (Dense)            (None, 50)                5050      
                                                                 
 dense_25 (Dense)            (None, 1)                 51        
                                                                 
Total params: 61001 (238.29 KB)
Trainable params: 61001 (238.29 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [130]:
early_stopping = EarlyStopping(monitor='loss', patience=3, restore_best_weights=True)
model.fit(X, y, epochs=200, batch_size=512, callbacks=[early_stopping])
y_pred = model.predict(df_test)

Epoch 1/200
349/349 [==============================] - 2s 5ms/step - loss: 704986535165952.0000
Epoch 2/200
349/349 [==============================] - 2s 5ms/step - loss: 704363697799168.0000
Epoch 3/200
349/349 [==============================] - 2s 5ms/step - loss: 702196953907200.0000
Epoch 4/200
349/349 [==============================] - 2s 5ms/step - loss: 697581072023552.0000
Epoch 5/200
349/349 [==============================] - 2s 5ms/step - loss: 689621893644288.0000
Epoch 6/200
349/349 [==============================] - 2s 5ms/step - loss: 677788856090624.0000
Epoch 7/200
349/349 [==============================] - 2s 5ms/step - loss: 661448451686400.0000
Epoch 8/200
349/349 [==============================] - 2s 5ms/step - loss: 641036686721024.0000
Epoch 9/200
349/349 [==============================] - 2s 5ms/step - loss: 616980910440448.0000
Epoch 10/200
349/349 [==============================] - 2s 5ms/step - loss: 589941071413248.0000
Epoch 11/200
349/349 [=================

In [131]:
test_pred = model.predict(X_test)
print("root_mean_squared_error: ",mean_squared_error(y_test,test_pred,squared=False))

1675/1675 [==============================] - 2s 1ms/step
root_mean_squared_error:  12736915.956930775


In [132]:
filepath = 'neural_2dec_2.csv'
with open(filepath, mode='w', newline='') as file: 
    writer = csv.writer(file)
    c=1
    writer.writerow(['row ID', 'price_doc'])
    for i in y_pred:
        writer.writerow([c ,i[0]])
        c=c+1

In [ ]:
 2573327.934773121
11256709.760698644
root_mean_squared_error:  12736915.956930775